In [3]:
import hail

import hailtop.batch as hb

backend = hb.ServiceBackend(billing_project = 'finucane-splicing-hj',
                            remote_tmpdir = 'gs://sqtl_gtex')

import pandas as pd

Explaination of script last updates 2 17 2024
--
This script creates a hail batch object that is able to send a job to a hail cluster. 

The jobs that we will be sending is the empirical bayes pipeline. 





run on simulations
--

In [4]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 25 #samples every 100 index



#scale to all lists of simulations
simulation_type=['single_1',
                'single_2',
                'double_1','double_2',
                 'triple_1', 'triple_2']
data_amt=['low', 'mid', 'high']
i=0
k=0
for i in range(len(simulation_type)):

    for k in range(len(data_amt)):
        l = b.new_job(name='all_data_ammounts')
        l.memory('20Gi')
        l.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
        tissue='test_tissue'
    #localizing the files
        input_script = "/home/run_optimizer_alt_ss_w_mixed_effects.py"    
        
        input_file_tissue = b.read_input("gs://sqtl_gtex/files/testing/mixture_testing/inputs_simulations/"+data_amt[k]+"_NVE_"+simulation_type[i]+"_counts.csv.gz")
        
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/testing/mixture_testing/inputs_simulations/filtering_exons_in_test_sample.csv.gz")
            
        
        for job_index in range(num_jobs):
            
            j = b.new_job(name='run_emp_bayes_test_')
            j.memory('20Gi')
            j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
        
            j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
        
            b.write_output(j.output, f'gs://sqtl_gtex/files/testing/mixture_testing/outputs_simulations_new_pipeline/{data_amt[k]}_{simulation_type[i]}_EF_FPR_5_alphas_betas-{job_index}.csv')
        
            
b.run()



    

Output()

Output()

batch 8239045 complete: success


try running new pipeline in a tissue
--

In [ ]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 100

#list_of_tissues=['Whole_Blood','Adipose_Subcutaneous']

list_of_tissues=['Esophagus_Mucosa','Thyroid']

b = hb.Batch(backend=backend, name='emp_bayes')

for tissue in list_of_tissues:
    
    
  
    for job_index in range(num_jobs):
    
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
       
        #localizing the files
        
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
            
        input_script = "/home/run_optimizer_alt_ss_w_mixed_effects.py"
    
        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
    
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
        
        j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
        
        b.write_output(j.output, 'gs://sqtl_gtex/files/testing/mixture_testing/real_data_outputs/' + tissue + '_alt_ss_EF_5_FPR_5_' + str(job_index) + '_alphas_betas.csv')

      # b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/alt_ss/optimizer_results/' + tissue + '_alt_ss_EF_5_FPR_5_' + str(job_index) +'_alphas_betas.csv')
    
b.run() 
    

Output()

Output()

other code
--

In [29]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 500 

j.memory('20Gi')

tissue=['Esophagus_Mucosa']

b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in ['Esophagus_Mucosa']:
    
    
    for job_index in range(5):
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
       
        #localizing the files
        
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
            
        input_script = "/home/run_optimizer_SE_w_mixed_effects.py"
    
        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
    
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/skipped_exons_w_gene_names_and_assigned_abc_introns.csv.gz")
        
        j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
    
        b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/SE/optimizer_results/' + tissue + '_SE_EF_5_FPR_5_' + str(job_index) +'alphas_betas.csv')
    b.run() 

Output()

Output()

batch 8236401 complete: failure


In [7]:
output_directory='/Users/hannahjacobs/MIT Dropbox/Hannah Jacobs/GradSchool/Finucane/splicing_variation_in_humans_2022/scripts/01_analysis_pipeline/00_run_emprical_bayes/emp_bayes_in_hail_pipeline-testing/data/mixture_implementation_testing/'

import pandas as pd

In [75]:
pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_100_iters.csv').params.iloc[0

SyntaxError: incomplete input (2429794027.py, line 1)

In [60]:
cluster_name=pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_100_iters.csv').cluster_name.iloc[0]
#intron_2=pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_300_iters.csv').intron_2.iloc[0]

In [61]:
pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_100_iters.csv').params.iloc[0]

'[6.57158227e-05 1.41314264e+00 5.43324086e+01 7.75423011e+00\n 5.99606529e-02 9.40039347e-01]'

In [62]:
#pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_300_iters_again.csv')

In [63]:


whole_blood = pd.read_csv('/Users/hannahjacobs/MIT Dropbox/Hannah Jacobs/GradSchool/Finucane/splicing_variation_in_humans_2022/GTEX sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/Whole_Blood_perind_numers.counts.gz',
                         low_memory=False,compression='gzip',
                          sep=' ', index_col=0)



In [64]:
cluster_name

'chr10:114160266:114162637:clu_9616_chr10:114157680:114162637:clu_9616'

In [65]:
k=whole_blood[whole_blood.index.isin(['chr10:114160266:114162637:clu_9616'])].values

N_minus_k=whole_blood[whole_blood.index.isin(['chr10:114157680:114162637:clu_9616'])].values

In [66]:
n=N_minus_k+k

In [ ]:
pd.DataFrame({'k':k[0], 'n': n[0]}).to_csv(output_directory+'K_N_of_confusing_exon.csv')

In [ ]:
import seaborn as sns

In [ ]:
psi=(k[0][n[0]>0])/(n[0][n[0]>0])

In [ ]:
sns.histplot(psi, bins=100)

In [69]:
#pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0_300_iters_again.csv')

In [25]:
list_of_LLs=pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0.csv').LogLikelihood.iloc[0]


In [41]:
pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0.csv').params_mixture.iloc[0]

'[3.46069345 0.35122382 3.07442231 1.61101392 0.06508526 0.93491474]'

In [47]:
#pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0.csv').function_output_emp_bayes.iloc[0]

In [42]:
pd.read_csv(output_directory+'Whole_Blood_alt_ss_EF_5_FPR_5_alphas_betas-0.csv')

'[-3002.1720710837444, -2731.773383662098, -2538.6194928874347, -2390.5870888872328, -2280.321107548892, -2208.7826811545706, -2181.457484176121, -2206.0948774713147, -2288.63247450775, -2428.6541246310517, -2619.1632426226433, -2858.467098694068, -3162.9555404440057, -3120.8830152957617, 32488.088088996625, -1319.9322287124648, -1326.5721579693982, -1331.2542226285916, -1336.3720316471515, -1341.7573062022402, -1347.3213130799513, -1352.9899590235873, -1358.6796889816976, -1364.3975360593226, -1370.0643366553609, -1375.6975865529475, -1381.2541132267913, -1386.7155203681295, -1392.0719108828644, -1397.306336904589, -1402.4062429848645, -1407.3666677550764, -1412.1911695567514, -1416.867342853135, -1421.3988230770979, -1425.7883691477805, -1430.0400439347252, -1434.1592503681277, -1438.1478477073952, -1442.0082498191514, -1445.7451159166158, -1449.3598263964536, -1452.8543092286895, -1456.2325591779672, -1459.5244671598512, -1462.7240503195137, -1465.8361247791076, -1468.8670320694364,

In [4]:
b = hb.Batch(backend=backend, name='emp_bayes_alt_ss')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('10Gi')
   
    #localizing the files
    

    
    
        
    input_script = "/home/run_optimizer_alt_ss.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/testing/EF_ouputs/alt_ss/optimizer_results/removed_prior/'+tissue+'_alt_ss_EF_5_FPR_5_alphas_betas.csv')

b.run() 

/opt/anaconda3/envs/running_hail_env/lib/python3.11/site-packages/rich/live.py:229: UserWarning:

install "ipywidgets" for Jupyter support

batch 8225675 complete: success


In [6]:
b = hb.Batch(backend=backend, name='assign_EF_alt_ss')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='assign_EF_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    
    input_script = "/home/assign_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/alt_ss/optimizer_results/"+tissue+"_alt_ss_alphas_betas.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/assigned_EFs/'+tissue+'_EFs_alt_ss.csv')

b.run() 

Output()

batch 7689646 complete: success


In [ ]:
b = hb.Batch(backend=backend, name='concat_EFs')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='concat_EFs'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/alt_ss/assigned_EFs/"+tissue+"_EFs_alt_ss.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 dfs = list() \
            for i, f in enumerate{tissue}: \
                data = pd.read_csv({tissue_file}) \
                data=data.assign(tissue={tissue}) \
                dfs.append(data)

                df = pd.concat(dfs, ignore_index=True)
                
                df.to_csv({j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/all_splicing_events.csv.gz')



In [37]:
b = hb.Batch(backend=backend, name='assign_EF_SE')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='assign_EF_'+tissue)
    j.memory('10Gi')
   
    #localizing the files
    
    input_script = "/home/assign_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/skipped_exons/optimizer_results/"+tissue+"_SE_alphas_betas.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/skipped_exons/assigned_EFs/'+tissue+'_EFs_SE.csv')

b.run() 

Output()

batch 8101537 complete: success


 0. test/debug with a single tissue


In [5]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in ['Whole_Blood']:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {5} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_5_percent_FPR_5_percent/'+tissue+'_alt_ss_EF_5_FPR_5_values.csv')

b.run() 
    

Output()


KeyboardInterrupt



1. Run across all tissues

In [7]:
data_directory='/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/splicing_variation_in_humans_2022/data/00_data/emp_bayes_input/'


alt_splice_sites=pd.read_csv(data_directory+'alt_splice_sites_filtered_for_popPSI.csv.gz', compression='gzip')





gtex_tissue_list=list(set(alt_splice_sites.tissue))

gtex_tissue_list

In [17]:
#psi_directory='/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/GTEx sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/'


run on all tissues EF 5%
-

In [21]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {0.05} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_5_percent_FPR_5_percent/'+tissue+'_alt_ss_EF_5_FPR_5_values.csv')

b.run() 
    

Output()

A transient error occured. We will automatically retry. Do not be alarmed. We have thus far seen 2 transient errors
(current delay: 0.2). The most recent error was <class 'asyncio.exceptions.TimeoutError'> .

batch 7597050 complete: cancelled


In [8]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {0.20} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_20_percent/'+tissue+'_alt_ss_EF_20_FPR_5_values.csv')

b.run() 
    

Output()

batch 7624031 complete: cancelled


In [ ]:
python3 run_EF.py \
                --tissue 'Adipose_Subcutaneous' \
                --tissue_file '/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/GTEX sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/Adipose_Subcutaneous_perind_numers.counts.gz' \
                --filter_file '/Users/hnjacobs/Dropbox (MIT)//GradSchool/Finuance/splicing_variation_in_humans_2022/data/alt_splice_sites_filtered_for_popPSI.csv.gz' \
                --output_file Adipose_output.csv.gz
    
    

In [ ]:
b = (backend=backend, name='emp_bayes')
for tissue in ['Adipose_Subcutaneous']:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_splice_sites_filtered_for_popPSI.csv.gz")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/'+tissue+'_EF_values.csv')

b.run() 
    